@Author: Girish <br>
@Date: 2024-09-08 <br>
@Last Modified by: Girish <br>
@Last Modified: 2024-09-08 <br>
@Title : Covid Data Analysis <br>

1

Death Percentage Globally

In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, round,when,avg

In [3]:

spark = SparkSession.builder.appName("Covid19Analysis").getOrCreate()

file_path = r"file:///D:\Apexon_python\Spark_Practice\covid_19_clean_complete.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)


result_df = (
    df.groupBy("Country/Region")
    .agg(
        sum(col("Confirmed").cast("int")).alias("Total_Confirmed"),
        sum(col("Deaths").cast("int")).alias("Total_Deaths"),
        round(
            (sum(col("Deaths").cast("int")) * 100.0) / sum(col("Confirmed").cast("int")),
            2
        ).alias("Death_percentage_globally")
    )
)
df.printSchema()
result_df.show()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+---------------+------------+-------------------------+
|Country/Region|Total_Confirmed|Total_Deaths|Death_percentage_globally|
+--------------+---------------+------------+-------------------------+
|          Chad|          64226|        5523|                      8.6|
|      Paraguay|         156373|        1663|                     1.06|
|        Russia|       45408411|      619385|                     1.36|
|         Yemen|          67180|       17707|                    26.36|
|       Senegal|         467457|        7177|                     1.54|
|    Cabo Verde| 

Dearh Percentage Locally

In [5]:


file_path_india = r"file:///D:\Apexon_python\Spark_Practice\covid_19_clean_complete.csv"
df = spark.read.csv(file_path_india, header=True, inferSchema=True)


result_df_india = (
    df.groupBy("Province/State")
    .agg(
        sum(col("Confirmed").cast("int")).alias("Total_Confirmed"),
        sum(col("Deaths").cast("int")).alias("Total_Deaths"),
        round(
            (sum(col("Deaths").cast("int")) * 100.0) / sum(col("Confirmed").cast("int")),
            2
        ).alias("Death_percentageLocally")
    )
)

df.printSchema()
result_df_india.show()







root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------------+---------------+------------+-----------------------+
|      Province/State|Total_Confirmed|Total_Deaths|Death_percentageLocally|
+--------------------+---------------+------------+-----------------------+
|            Manitoba|          35148|         746|                   2.12|
|            Anguilla|            361|           0|                    0.0|
|           Guangdong|         268051|        1273|                   0.47|
|               Hunan|         178641|         662|                   0.37|
|              Shanxi|          30441|           0|          

2

Infected Polpulation Locally

In [8]:
file_path_india = r"file:///D:\Apexon_python\Spark_Practice\covid_19_india.csv"
df = spark.read.csv(file_path_india, header=True, inferSchema=True)



df = df.withColumn("Confirmed", col("Confirmed").cast("int")) .withColumn("Cured", col("Cured").cast("int"))


result_df = (
    df
    .groupBy("State/UnionTerritory")
    .agg(
        sum("Confirmed").alias("Total_Confirmed"),
        sum("Cured").alias("Total_Cured")
    )
    .withColumn(
        "locally_Infected",
        when(col("Total_Confirmed") > 0,
             100 - (col("Total_Cured") * 100.0 / col("Total_Confirmed"))
        ).otherwise(None)
    )
)

result_df.printSchema()
result_df.show(truncate=False)

root
 |-- State/UnionTerritory: string (nullable = true)
 |-- Total_Confirmed: long (nullable = true)
 |-- Total_Cured: long (nullable = true)
 |-- locally_Infected: double (nullable = true)

+----------------------------------------+---------------+-----------+------------------+
|State/UnionTerritory                    |Total_Confirmed|Total_Cured|locally_Infected  |
+----------------------------------------+---------------+-----------+------------------+
|Nagaland                                |5041742        |4519526    |10.357848537271437|
|Karnataka                               |485970693      |441844360  |9.080039935659244 |
|Odisha                                  |160130533      |150923455  |5.749732938189865 |
|Kerala                                  |458906023      |420174235  |8.440026074793963 |
|Ladakh                                  |4054293        |3758960    |7.284451321105806 |
|Dadra and Nagar Haveli and Daman and Diu|1938632        |1841750    |4.997441494827285 

Infected Population GLobally

In [9]:
spark = SparkSession.builder.appName("Covid19Analysis").getOrCreate()

file_path_india = r"file:///D:\Apexon_python\Spark_Practice\covid_19_india.csv"
df = spark.read.csv(file_path_india, header=True, inferSchema=True)




result_df = df.groupBy("State/UnionTerritory") \
    .agg(
        sum("Confirmed").alias("Total_Cases"),
        sum("ConfirmedIndianNational").alias("Total_Active")
    ) \
    .withColumn(
        "Globally_Infected_percentage",
        (col("Total_Active") * 100.0) / col("Total_Cases")
    )



df.printSchema()
result_df.show()

root
 |-- Sno: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- State/UnionTerritory: string (nullable = true)
 |-- ConfirmedIndianNational: string (nullable = true)
 |-- ConfirmedForeignNational: string (nullable = true)
 |-- Cured: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Confirmed: integer (nullable = true)

+--------------------+-----------+------------+----------------------------+
|State/UnionTerritory|Total_Cases|Total_Active|Globally_Infected_percentage|
+--------------------+-----------+------------+----------------------------+
|            Nagaland|    5041742|        null|                        null|
|           Karnataka|  485970693|       405.0|        8.333835884214524E-5|
|              Odisha|  160130533|        23.0|        1.436328198570350...|
|              Kerala|  458906023|      1091.0|        2.377393072480986E-4|
|              Ladakh|    4054293|       162.0|         0.003995764

3

Countries with Heighest Infection rates

In [10]:

file_path_india = r"file:///D:\Apexon_python\Spark_Practice\covid_19_clean_complete.csv"
df = spark.read.csv(file_path, header=True, inferSchema=True)




result_df = df.withColumn("confirmed", col("Confirmed").cast("integer")) \
    .groupBy("Country/Region") \
    .agg(
        sum("confirmed").alias("Total_Confirmed")
    ) \
    .orderBy(col("Total_Confirmed").desc())

df.printSchema()
result_df.show()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+---------------+
|Country/Region|Total_Confirmed|
+--------------+---------------+
|            US|      224345948|
|        Brazil|       89524967|
|        Russia|       45408411|
|         India|       40883464|
|         Spain|       27404045|
|United Kingdom|       26748587|
|         Italy|       26745145|
|        France|       21210926|
|       Germany|       21059152|
|          Iran|       19339267|
|          Peru|       19263916|
|        Turkey|       17903345|
|         Chile|       16935654|
|        Mexico|       14946202|
|         China|       14132002|

4

To find out the countries ans continents witht eh heighest death counts

In [11]:

file_path_india = r"file:///D:\Apexon_python\Spark_Practice\covid_19_india.csv"
df = spark.read.csv(file_path_india, header=True, inferSchema=True)

df.printSchema()

result_df = (df.groupBy("State/UnionTerritory")  
             .agg(sum("Deaths").alias("Country_highest_Deaths"))  
             .orderBy(col("Country_highest_Deaths").desc()))  

result_df.show()

root
 |-- Sno: integer (nullable = true)
 |-- Date: date (nullable = true)
 |-- Time: string (nullable = true)
 |-- State/UnionTerritory: string (nullable = true)
 |-- ConfirmedIndianNational: string (nullable = true)
 |-- ConfirmedForeignNational: string (nullable = true)
 |-- Cured: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Confirmed: integer (nullable = true)

+--------------------+----------------------+
|State/UnionTerritory|Country_highest_Deaths|
+--------------------+----------------------+
|         Maharashtra|              23737432|
|           Karnataka|               6053762|
|          Tamil Nadu|               5916658|
|               Delhi|               4943294|
|       Uttar Pradesh|               4143450|
|         West Bengal|               3846989|
|      Andhra Pradesh|               2939367|
|              Punjab|               2785594|
|             Gujarat|               2219448|
|        Chhattisgarh|               2063920|
|       

5

Countries with the avg number of deaths by day

In [14]:



file_path = r"file:///D:\Apexon_python\Spark_Practice\covid_19_clean_complete.csv"


df = spark.read.csv(file_path, header=True, inferSchema=True)
df.printSchema()

daily_deaths_df = (df.groupBy("Country/Region", "Date")
                   .agg(sum("Deaths").alias("Daily_Deaths")))

avg_daily_deaths_df = (daily_deaths_df.groupBy("Country/Region")
                       .agg(avg("Daily_Deaths").alias("Avg_Daily_Deaths"))
                       .orderBy(col("Avg_Daily_Deaths").desc()))

avg_daily_deaths_df.show()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+------------------+
|Country/Region|  Avg_Daily_Deaths|
+--------------+------------------+
|            US|58571.335106382976|
|United Kingdom|21264.760638297874|
|        Brazil|20946.989361702126|
|         Italy| 19721.89893617021|
|        France|16215.553191489362|
|         Spain| 16133.13829787234|
|        Mexico| 9192.962765957447|
|         India| 5913.994680851064|
|          Iran| 5447.531914893617|
|       Belgium| 5125.952127659574|
|       Germany| 4634.691489361702|
|        Canada| 3721.095744680851|
|         China|3576.6648936170213|
|          Peru| 

Continents with Avg number of deaths by day

In [15]:
file_path = r"file:///D:\Apexon_python\Spark_Practice\covid_19_clean_complete.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.printSchema()

daily_deaths_by_continent_df = (df.groupBy("WHO Region", "Date")
                                .agg(sum("Deaths").alias("Daily_Deaths")))

avg_daily_deaths_df = (daily_deaths_by_continent_df.groupBy("WHO Region")
                       .agg(avg("Daily_Deaths").alias("Avg_Daily_Deaths"))
                       .withColumnRenamed("WHO Region", "Continent")  # Renaming the column
                       .orderBy(col("Avg_Daily_Deaths").desc()))


avg_daily_deaths_df.show()

root
 |-- Province/State: string (nullable = true)
 |-- Country/Region: string (nullable = true)
 |-- Lat: double (nullable = true)
 |-- Long: double (nullable = true)
 |-- Date: date (nullable = true)
 |-- Confirmed: integer (nullable = true)
 |-- Deaths: integer (nullable = true)
 |-- Recovered: integer (nullable = true)
 |-- Active: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------------+------------------+
|           Continent|  Avg_Daily_Deaths|
+--------------------+------------------+
|            Americas| 102974.9574468085|
|              Europe|102505.53191489361|
|Eastern Mediterra...|10234.196808510638|
|     South-East Asia| 7756.031914893617|
|     Western Pacific| 4959.734042553191|
|              Africa| 2340.308510638298|
+--------------------+------------------+



6

Average of cases divided by the number of population of each country

In [16]:
file_path = "file:///D:/Apexon_python/Spark_Practice/worldometer_data.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True)

df.printSchema()

df_with_cases_per_population = df.withColumn(
    "Cases_Per_Population", 
    col("TotalCases") * 1.0 / col("Population")
)

avg_cases_per_population_df = (df_with_cases_per_population.groupBy("Country/Region")
                                .agg(avg("Cases_Per_Population").alias("Avg_Cases_Per_Population"))
                                .orderBy(col("Avg_Cases_Per_Population").desc()))

avg_cases_per_population_df.show(10)

spark.stop()

root
 |-- Country/Region: string (nullable = true)
 |-- Continent: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- TotalCases: integer (nullable = true)
 |-- NewCases: integer (nullable = true)
 |-- TotalDeaths: integer (nullable = true)
 |-- NewDeaths: integer (nullable = true)
 |-- TotalRecovered: integer (nullable = true)
 |-- NewRecovered: integer (nullable = true)
 |-- ActiveCases: integer (nullable = true)
 |-- Serious,Critical: integer (nullable = true)
 |-- Tot Cases/1M pop: integer (nullable = true)
 |-- Deaths/1M pop: double (nullable = true)
 |-- TotalTests: integer (nullable = true)
 |-- Tests/1M pop: integer (nullable = true)
 |-- WHO Region: string (nullable = true)

+--------------+------------------------+
|Country/Region|Avg_Cases_Per_Population|
+--------------+------------------------+
|         Qatar|    0.039921575750452756|
| French Guiana|    0.027145648579588157|
|       Bahrain|     0.02513023907975126|
|    San Marino|    0.02059638163